In [4]:
!pip install supervision ultralytics opencv-python numpy
!pip install json datetime
!pip install supervision ultralytics opencv-python numpy
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

### Test Code 1

In [ ]:
import supervision as sv
from ultralytics import YOLO
import cv2
import numpy as np
import json
from datetime import datetime
import os

# ===== Configuration =====
SOURCE_VIDEO_PATH = "parking_crop.mp4"  # Your parking lot video
TARGET_VIDEO_PATH = "parking_output.mp4"
JSON_OUTPUT_PATH = "parking_data.json"

# Define parking slots as polygons (example coordinates - adjust to your video)
parking_slots = {
    1: np.array([(100, 100), (200, 100), (200, 200), (100, 200)], np.int32),
    2: np.array([(220, 100), (320, 100), (320, 200), (220, 200)], np.int32),
    # Add more slots as needed
}

# Vehicle classes in COCO dataset (car, motorcycle, truck, bus)
VEHICLE_CLASSES = [2, 3, 5, 7]

# ===== Initialize =====
occupancy = {slot_id: False for slot_id in parking_slots}
vehicle_count = 0
parking_data = {}

# ===== Helper Functions =====
def is_vehicle_in_slot(vehicle_bbox, slot_polygon):
    """Check if a vehicle is parked in a slot"""
    # Get vehicle center point
    x1, y1, x2, y2 = vehicle_bbox
    center = ((x1 + x2) / 2, (y1 + y2) / 2)

    # Check if center is within the polygon
    return cv2.pointPolygonTest(slot_polygon, center, False) >= 0

# ===== Main Processing =====
model = YOLO("yolov8s.pt")  # or yolov8x.pt for better accuracy
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for frame_number, result in enumerate(
        model.track(source=SOURCE_VIDEO_PATH, tracker="bytetrack.yaml", show=False, stream=True, persist=True)
    ):
        frame = result.orig_img

        # Convert results to supervision Detections object
        detections = sv.Detections(
            xyxy=result.boxes.xyxy.cpu().numpy(),
            confidence=result.boxes.conf.cpu().numpy(),
            class_id=result.boxes.cls.cpu().numpy().astype(int),
            tracker_id=result.boxes.id.cpu().numpy().astype(int) if result.boxes.id is not None else None
        )

        # Reset current frame occupancy
        current_frame_occupancy = {slot_id: False for slot_id in parking_slots}

        # Process detections
        for i, (bbox, conf, class_id, tracker_id) in enumerate(zip(
            detections.xyxy,
            detections.confidence,
            detections.class_id,
            detections.tracker_id if detections.tracker_id is not None else [None]*len(detections)
        )):
            if class_id in VEHICLE_CLASSES:
                # Check which parking slot the vehicle is in
                for slot_id, slot_polygon in parking_slots.items():
                    if is_vehicle_in_slot(bbox, slot_polygon):
                        current_frame_occupancy[slot_id] = True

                        # Initialize vehicle data if new
                        if tracker_id not in parking_data:
                            parking_data[tracker_id] = {
                                "slot_id": slot_id,
                                "entry_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                                "entry_frame": frame_number,
                                "vehicle_class": int(class_id),
                                "confidence": float(conf),
                                "bbox_history": []
                            }

                        # Update bbox history
                        parking_data[tracker_id]["bbox_history"].append({
                            "frame_number": frame_number,
                            "bbox": [float(x) for x in bbox],
                            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        })

                        break

        # Update occupancy and detect departures
        for slot_id, occupied in current_frame_occupancy.items():
            if occupied and not occupancy[slot_id]:
                # Vehicle arrived in this slot
                vehicle_count += 1
            elif not occupied and occupancy[slot_id]:
                # Vehicle left this slot
                pass  # Could add departure time tracking here

            occupancy[slot_id] = occupied

        # Draw parking slots and status
        for slot_id, slot_polygon in parking_slots.items():
            color = (0, 255, 0) if not occupancy[slot_id] else (0, 0, 255)  # Green=free, Red=occupied
            cv2.polylines(frame, [slot_polygon], isClosed=True, color=color, thickness=2)
            cv2.putText(frame, f"Slot {slot_id}", tuple(slot_polygon[0]),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        # Display counts
        cv2.putText(frame, f"Total Vehicles: {vehicle_count}", (50, 50),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Write frame to output video
        sink.write_frame(frame)

# Save results
json_output = {
    "parking_slots": len(parking_slots),
    "total_vehicles": vehicle_count,
    "current_occupancy": occupancy,
    "vehicle_data": parking_data
}

with open(JSON_OUTPUT_PATH, "w") as f:
    json.dump(json_output, f, indent=4)

print(f"Processing complete. Results saved to {TARGET_VIDEO_PATH} and {JSON_OUTPUT_PATH}")


video 1/1 (frame 1/849) /content/parking_crop.mp4: 640x544 2 cups, 16.7ms
video 1/1 (frame 2/849) /content/parking_crop.mp4: 640x544 2 cups, 16.0ms
video 1/1 (frame 3/849) /content/parking_crop.mp4: 640x544 2 cups, 16.0ms
video 1/1 (frame 4/849) /content/parking_crop.mp4: 640x544 2 cups, 16.0ms
video 1/1 (frame 5/849) /content/parking_crop.mp4: 640x544 2 cups, 16.0ms
video 1/1 (frame 6/849) /content/parking_crop.mp4: 640x544 2 cups, 16.0ms
video 1/1 (frame 7/849) /content/parking_crop.mp4: 640x544 2 cups, 15.9ms
video 1/1 (frame 8/849) /content/parking_crop.mp4: 640x544 2 cups, 16.0ms
video 1/1 (frame 9/849) /content/parking_crop.mp4: 640x544 2 cups, 11.3ms
video 1/1 (frame 10/849) /content/parking_crop.mp4: 640x544 2 cups, 11.4ms
video 1/1 (frame 11/849) /content/parking_crop.mp4: 640x544 2 cups, 11.3ms
video 1/1 (frame 12/849) /content/parking_crop.mp4: 640x544 2 cups, 11.3ms
video 1/1 (frame 13/849) /content/parking_crop.mp4: 640x544 2 cups, 11.3ms
video 1/1 (frame 14/849) /content

### Test Code 2

In [ ]:
import supervision as sv
from ultralytics import YOLO
import cv2
import numpy as np
import json
from datetime import datetime
import os

# ===== Configuration =====
SOURCE_VIDEO_PATH = "parking_crop.mp4"
TARGET_VIDEO_PATH = "parking_output.mp4"
JSON_OUTPUT_PATH = "parking_data.json"
CONFIDENCE_THRESHOLD = 0.3  # Lowered threshold for better detection
IOU_THRESHOLD = 0.5  # Intersection over Union threshold
DEBUG_MODE = True  # Set to True to save debug images

# Define parking slots (using the coordinates provided)
parking_slots = {
    1: np.array([[203, 55], [341, 53], [338, 114], [206, 115]], np.int32),
    2: np.array([[207, 119], [336, 115], [333, 176], [199, 173]], np.int32),
    3: np.array([[208, 175], [331, 176], [338, 233], [206, 233]], np.int32),
    4: np.array([[205, 236], [341, 226], [341, 287], [211, 288]], np.int32),
    5: np.array([[209, 297], [344, 294], [350, 353], [207, 353]], np.int32),
    6: np.array([[206, 302], [69, 297], [62, 345], [210, 348]], np.int32),
    7: np.array([[200, 282], [76, 288], [68, 235], [199, 239]], np.int32),
    8: np.array([[200, 226], [96, 233], [81, 174], [200, 176]], np.int32),
    9: np.array([[192, 167], [83, 170], [82, 124], [204, 120]], np.int32),
    10: np.array([[200, 98], [59, 106], [76, 58], [191, 60]], np.int32),
}

# Expanded vehicle classes (car, motorcycle, bus, truck, etc.)
VEHICLE_CLASSES = [1, 2, 3, 5, 7]  # bicycle, car, motorcycle, bus, truck

# ===== Initialize =====
occupancy = {slot_id: False for slot_id in parking_slots}
vehicle_count = 0
parking_data = {}
track_history = {}
os.makedirs("debug_frames", exist_ok=True)

# Initialize the byte tracker
byte_tracker = sv.ByteTrack()

# ===== Helper Functions =====
def is_vehicle_in_slot(vehicle_bbox, slot_polygon):
    """Improved vehicle-slot matching using polygon intersection"""
    x1, y1, x2, y2 = vehicle_bbox
    vehicle_polygon = np.array([[x1, y1], [x2, y1], [x2, y2], [x1, y2]])

    # Calculate intersection area
    intersection_area = cv2.intersectConvexConvex(vehicle_polygon, slot_polygon)[0]
    vehicle_area = (x2 - x1) * (y2 - y1)

    # Considered in slot if > 30% overlap
    return (intersection_area / vehicle_area) > 0.3

def draw_parking_slots(frame, slots, occupancy):
    """Enhanced visualization with slot status"""
    for slot_id, polygon in slots.items():
        # Semi-transparent fill
        overlay = frame.copy()
        color = (0, 200, 0) if not occupancy[slot_id] else (0, 0, 200)
        cv2.fillPoly(overlay, [polygon], color)
        cv2.addWeighted(overlay, 0.3, frame, 0.7, 0, frame)

        # Outline
        cv2.polylines(frame, [polygon], isClosed=True, color=(255,255,255), thickness=2)

        # Slot number at centroid
        centroid = np.mean(polygon, axis=0).astype(int)
        cv2.putText(frame, f"{slot_id}", (centroid[0]-10, centroid[1]),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

def save_debug_frame(frame, frame_number, detections):
    """Save frame with detection info for debugging"""
    debug_frame = frame.copy()

    # Draw all detections (regardless of class)
    for detection in detections:
        x1, y1, x2, y2 = detection[0:4].astype(int)
        cv2.rectangle(debug_frame, (x1, y1), (x2, y2), (200, 200, 0), 2)

    cv2.imwrite(f"debug_frames/frame_{frame_number:04d}.jpg", debug_frame)

# ===== Main Processing =====
model = YOLO("yolov8m.pt")  # Using medium model for better accuracy
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(TARGET_VIDEO_PATH, fourcc, fps, (frame_width, frame_height))

frame_number = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_number += 1
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Run YOLOv8 detection
    results = model.track(
        frame,
        persist=True,
        conf=CONFIDENCE_THRESHOLD,
        iou=IOU_THRESHOLD,
        classes=VEHICLE_CLASSES
    )

    # Convert to supervision Detections
    detections = sv.Detections.from_ultralytics(results[0])

    # Update tracker
    detections = byte_tracker.update_with_detections(detections)

    if DEBUG_MODE:
        save_debug_frame(frame, frame_number, detections.xyxy)

    # Reset current frame occupancy
    current_frame_occupancy = {slot_id: False for slot_id in parking_slots}
    current_vehicles = set()

    # Process each detection
    for box, confidence, class_id, tracker_id in zip(
        detections.xyxy,
        detections.confidence,
        detections.class_id,
        detections.tracker_id
    ):
        if tracker_id is None:
            continue

        # Check parking slot occupancy
        for slot_id, slot_polygon in parking_slots.items():
            if is_vehicle_in_slot(box, slot_polygon):
                current_frame_occupancy[slot_id] = True
                current_vehicles.add(tracker_id)

                # Initialize new vehicle tracking
                if tracker_id not in parking_data:
                    parking_data[tracker_id] = {
                        "slot_id": slot_id,
                        "entry_time": current_time,
                        "entry_frame": frame_number,
                        "vehicle_class": int(class_id),
                        "class_name": model.names[int(class_id)],
                        "confidence": float(confidence),
                        "bbox_history": []
                    }
                    vehicle_count += 1

                # Update tracking history
                parking_data[tracker_id]["bbox_history"].append({
                    "frame_number": frame_number,
                    "bbox": [float(x) for x in box],
                    "timestamp": current_time,
                    "current_slot": slot_id
                })

                # Draw bounding box with class info
                label = f"{tracker_id}: {model.names[int(class_id)]} {confidence:.2f}"
                cv2.rectangle(frame, (int(box[0]), int(box[1])),
                            (int(box[2]), int(box[3])), (0, 255, 255), 2)
                cv2.putText(frame, label, (int(box[0]), int(box[1])-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
                break

    # Update occupancy status
    for slot_id, occupied in current_frame_occupancy.items():
        occupancy[slot_id] = occupied

    # Draw parking slots and info
    draw_parking_slots(frame, parking_slots, occupancy)

    # Display info overlay
    occupied_slots = sum(occupancy.values())
    info_text = [
        f"Frame: {frame_number}/{total_frames}",
        f"Time: {current_time}",
        f"Vehicles: {len(current_vehicles)} (Total: {vehicle_count})",
        f"Occupancy: {occupied_slots}/{len(parking_slots)}",
        f"Detections: {len(detections)}"
    ]

    for i, text in enumerate(info_text):
        cv2.putText(frame, text, (20, 30 + i*30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Write frame to output
    out.write(frame)

    # Console log
    print(f"Frame {frame_number:04d} | Vehicles: {len(current_vehicles):02d} | Occupied: {occupied_slots:02d}/{len(parking_slots):02d}")

# Release resources
cap.release()
out.release()

# Save results
json_output = {
    "processing_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "video_source": SOURCE_VIDEO_PATH,
    "video_info": {
        "width": frame_width,
        "height": frame_height,
        "fps": fps,
        "total_frames": total_frames
    },
    "parking_config": {
        "total_slots": len(parking_slots),
        "slot_coordinates": {k: v.tolist() for k, v in parking_slots.items()}
    },
    "detection_settings": {
        "confidence_threshold": CONFIDENCE_THRESHOLD,
        "iou_threshold": IOU_THRESHOLD,
        "vehicle_classes": VEHICLE_CLASSES
    },
    "results": {
        "total_vehicles": vehicle_count,
        "final_occupancy": occupancy,
        "vehicle_data": parking_data
    }
}

with open(JSON_OUTPUT_PATH, "w") as f:
    json.dump(json_output, f, indent=4)

print(f"\nProcessing complete. Results saved to:")
print(f"- Video output: {TARGET_VIDEO_PATH}")
print(f"- JSON data: {JSON_OUTPUT_PATH}")
if DEBUG_MODE:
    print(f"- Debug frames: debug_frames/")

100%|██████████| 49.7M/49.7M [00:00<00:00, 139MB/s]



0: 640x544 (no detections), 98.6ms
Speed: 6.2ms preprocess, 98.6ms inference, 99.8ms postprocess per image at shape (1, 3, 640, 544)
Frame 0001 | Vehicles: 00 | Occupied: 00/10

0: 640x544 (no detections), 36.1ms
Speed: 3.4ms preprocess, 36.1ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 544)
Frame 0002 | Vehicles: 00 | Occupied: 00/10

0: 640x544 (no detections), 41.3ms
Speed: 3.7ms preprocess, 41.3ms inference, 13.5ms postprocess per image at shape (1, 3, 640, 544)
Frame 0003 | Vehicles: 00 | Occupied: 00/10

0: 640x544 (no detections), 55.8ms
Speed: 14.4ms preprocess, 55.8ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 544)
Frame 0004 | Vehicles: 00 | Occupied: 00/10

0: 640x544 (no detections), 50.8ms
Speed: 16.9ms preprocess, 50.8ms inference, 3.8ms postprocess per image at shape (1, 3, 640, 544)
Frame 0005 | Vehicles: 00 | Occupied: 00/10

0: 640x544 (no detections), 45.8ms
Speed: 9.9ms preprocess, 45.8ms inference, 1.6ms postprocess per image at s

### Final Code

In [ ]:
# 1. Completely remove problematic packages
!pip uninstall -y numpy opencv-python-headless supervision ultralytics

# 2: Reinstall OpenCV and supervision to make sure they link correctly
!pip install numpy==1.26.4
!pip install --upgrade --force-reinstall shapely supervision opencv-python-headless ultralytics

# 3. Install with explicit versions
!pip install numpy==1.23.5 --no-cache-dir
!pip install opencv-python==4.7.0.72
!pip install supervision==0.14.0
!pip install ultralytics==8.0.196
!pip install shapely==2.0.1

# 4. Verify installation
!python -c "import numpy; print(f'NumPy {numpy.__version__} OK')"

In [ ]:
import supervision as sv
from ultralytics import YOLO
import cv2
import numpy as np
import json
from datetime import datetime
import os
from shapely.geometry import Polygon
from shapely.errors import TopologicalError
import sys

# ===== Verify NumPy =====
try:
    print("NumPy version:", np.__version__)
except Exception as e:
    print(" NumPy issue:", e)
    sys.exit()

# ===== Configuration =====
SOURCE_VIDEO_PATH = "parking_crop.mp4"
TARGET_VIDEO_PATH = "parking_output.mp4"
JSON_OUTPUT_PATH = "parking_data.json"
CONFIDENCE_THRESHOLD = 0.3
IOU_THRESHOLD = 0.5
DEBUG_MODE = True

parking_slots = {
    1: np.array([[203, 55], [341, 53], [338, 114], [206, 115]], np.int32),
    2: np.array([[207, 119], [336, 115], [333, 176], [199, 173]], np.int32),
    3: np.array([[208, 175], [331, 176], [338, 233], [206, 233]], np.int32),
    4: np.array([[205, 236], [341, 226], [341, 287], [211, 288]], np.int32),
    5: np.array([[209, 297], [344, 294], [350, 353], [207, 353]], np.int32),
    6: np.array([[206, 302], [69, 297], [62, 345], [210, 348]], np.int32),
    7: np.array([[200, 282], [76, 288], [68, 235], [199, 239]], np.int32),
    8: np.array([[200, 226], [96, 233], [81, 174], [200, 176]], np.int32),
    9: np.array([[192, 167], [83, 170], [82, 124], [204, 120]], np.int32),
    10: np.array([[200, 98], [59, 106], [76, 58], [191, 60]], np.int32),
}

VEHICLE_CLASSES = [1, 2, 3, 5, 7]
# VEHICLE_CLASSES = None  # Detect all classes for debugging

occupancy = {slot_id: False for slot_id in parking_slots}
vehicle_count = 0
parking_data = {}
os.makedirs("debug_frames", exist_ok=True)

# ===== Initialization =====
model = YOLO("yolov8m.pt")
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
if not cap.isOpened():
    print(" Failed to open video.")
    sys.exit()

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
out = cv2.VideoWriter(TARGET_VIDEO_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))
byte_tracker = sv.ByteTrack()

def is_vehicle_in_slot(vehicle_bbox, slot_polygon):
    try:
        x1, y1, x2, y2 = map(float, vehicle_bbox)
        vehicle_poly = Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)])
        slot_poly = Polygon(slot_polygon)
        if not vehicle_poly.is_valid or not slot_poly.is_valid:
            return False
        intersection_area = vehicle_poly.intersection(slot_poly).area
        return (intersection_area / vehicle_poly.area) > 0.3
    except Exception as e:
        if DEBUG_MODE:
            print(f" Intersection error: {e}")
        return False

def draw_parking_slots(frame, slots, occupancy):
    for slot_id, polygon in slots.items():
        overlay = frame.copy()
        color = (0, 200, 0) if not occupancy[slot_id] else (0, 0, 200)
        cv2.fillPoly(overlay, [polygon], color)
        cv2.addWeighted(overlay, 0.3, frame, 0.7, 0, frame)
        cv2.polylines(frame, [polygon], isClosed=True, color=(255,255,255), thickness=2)
        centroid = np.mean(polygon, axis=0).astype(int)
        cv2.putText(frame, f"{slot_id}", (centroid[0]-10, centroid[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

def save_debug_frame(frame, frame_number, boxes):
    debug_frame = frame.copy()
    for box in boxes:
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(debug_frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
    cv2.imwrite(f"debug_frames/frame_{frame_number:04d}.jpg", debug_frame)

# ===== Main Loop =====
frame_number = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_number += 1
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    result = model.track(source=frame, persist=True, conf=CONFIDENCE_THRESHOLD, iou=IOU_THRESHOLD, classes=VEHICLE_CLASSES, verbose=False)
    if result is None or result[0].boxes is None:
        out.write(frame)
        continue

    print(f" Frame {frame_number}: Detected {len(result[0].boxes)} objects.")
    print("   Raw classes:", result[0].boxes.cls.cpu().numpy())

    detections = sv.Detections.from_ultralytics(result[0])
    detections = byte_tracker.update_with_detections(detections)

    if DEBUG_MODE:
        save_debug_frame(frame, frame_number, detections.xyxy)

    current_frame_occupancy = {slot_id: False for slot_id in parking_slots}
    current_vehicles = set()

    for box, conf, class_id, tracker_id in zip(detections.xyxy, detections.confidence, detections.class_id, detections.tracker_id):
        if tracker_id is None:
            continue
        for slot_id, polygon in parking_slots.items():
            if is_vehicle_in_slot(box, polygon):
                current_frame_occupancy[slot_id] = True
                current_vehicles.add(tracker_id)
                if tracker_id not in parking_data:
                    parking_data[tracker_id] = {
                        "slot_id": int(slot_id),
                        "entry_time": current_time,
                        "entry_frame": int(frame_number),
                        "vehicle_class": int(class_id),
                        "class_name": model.names[int(class_id)],
                        "confidence": float(conf),
                        "bbox_history": []
                    }
                    vehicle_count += 1
                parking_data[tracker_id]["bbox_history"].append({
                    "frame_number": int(frame_number),
                    "bbox": [float(x) for x in box],
                    "timestamp": current_time,
                    "current_slot": int(slot_id)
                })
                label = f"{tracker_id}: {model.names[int(class_id)]} {conf:.2f}"
                cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 255), 2)
                cv2.putText(frame, label, (int(box[0]), int(box[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
                break

    for slot_id in parking_slots:
        occupancy[slot_id] = current_frame_occupancy[slot_id]

    draw_parking_slots(frame, parking_slots, occupancy)
    occupied_slots = sum(occupancy.values())

    for i, text in enumerate([
        f"Frame: {frame_number}/{total_frames}",
        f"Time: {current_time}",
        f"Vehicles: {len(current_vehicles)} (Total: {vehicle_count})",
        f"Occupancy: {occupied_slots}/{len(parking_slots)}",
        f"Detections: {len(detections)}"
    ]):
        cv2.putText(frame, text, (20, 30 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    out.write(frame)
    print(f" Frame {frame_number} done | Vehicles: {len(current_vehicles)} | Occupied slots: {occupied_slots}")

# ===== Finalize =====
cap.release()
out.release()

# Convert all NumPy types to native Python types before JSON serialization
def convert_to_python_types(data):
    if isinstance(data, (np.integer, np.int64)):
        return int(data)
    elif isinstance(data, (np.floating, np.float32, np.float64)):
        return float(data)
    elif isinstance(data, np.ndarray):
        return data.tolist()
    elif isinstance(data, dict):
        return {convert_to_python_types(k): convert_to_python_types(v) for k, v in data.items()}
    elif isinstance(data, (list, tuple)):
        return [convert_to_python_types(item) for item in data]
    return data

with open(JSON_OUTPUT_PATH, "w") as f:
    json.dump(convert_to_python_types({
        "processing_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "video_source": SOURCE_VIDEO_PATH,
        "video_info": {
            "width": frame_width,
            "height": frame_height,
            "fps": fps,
            "total_frames": total_frames
        },
        "parking_config": {
            "total_slots": len(parking_slots),
            "slot_coordinates": {k: v.tolist() for k, v in parking_slots.items()}
        },
        "detection_settings": {
            "confidence_threshold": CONFIDENCE_THRESHOLD,
            "iou_threshold": IOU_THRESHOLD,
            "vehicle_classes": VEHICLE_CLASSES
        },
        "results": {
            "total_vehicles": vehicle_count,
            "final_occupancy": occupancy,
            "vehicle_data": parking_data
        }
    }), f, indent=4)

print(f"\n DONE: Output saved to:")
print(f" {TARGET_VIDEO_PATH}")
print(f" {JSON_OUTPUT_PATH}")
if DEBUG_MODE:
    print(f" Debug frames: debug_frames/")